###Задание по программированию: Линейная регрессия: прогноз оклада по описанию вакансии


использовать линейную регрессию

применять линейную регрессию к текстовым данным

In [ ]:
#sklearn.feature_extraction.text.TfidfVectorizer
#sklearn.linear_model.Ridge
#one-hot-кодирование

In [ ]:
# Code to read csv file into Colaboratory:!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [ ]:
link = 'https://drive.google.com/file/d/1cG13vdHqv2vgoXXepxfiUKLxd0XALWDb/view?usp=sharing' # The shareable link

In [ ]:
import pandas as pd

In [ ]:
downloaded = drive.CreateFile({'id':'1cG13vdHqv2vgoXXepxfiUKLxd0XALWDb'}) 
downloaded.GetContentFile('salary-train.csv')  
X_train = pd.read_csv('salary-train.csv')# Dataset is now stored in a Pandas Dataframe

In [ ]:
link_test = 'https://drive.google.com/file/d/1gpcgblUDx3YzFpWsUKyAxZ_bXXEdpNl2/view?usp=sharing'

In [ ]:
downloaded = drive.CreateFile({'id':'1gpcgblUDx3YzFpWsUKyAxZ_bXXEdpNl2'}) 
downloaded.GetContentFile('salary-test-mini.csv')  
X_test = pd.read_csv('salary-test-mini.csv')# Dataset is now stored in a Pandas Dataframe

In [ ]:
y_train = X_train['SalaryNormalized']

In [ ]:
X_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,International Sales Manager London ****k ****...,London,permanent,33000
1,An ideal opportunity for an individual that ha...,London,permanent,50000
2,Online Content and Brand Manager// Luxury Reta...,South East London,permanent,40000
3,A great local marketleader is seeking a perman...,Dereham,permanent,22500
4,Registered Nurse / RGN Nursing Home for Young...,Sutton Coldfield,NaN,20355


In [ ]:
X_test.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,We currently have a vacancy for an HR Project ...,Milton Keynes,contract,NaN
1,A Web developer opportunity has arisen with an...,Manchester,permanent,NaN


In [ ]:
X_train.shape

(60000, 4)

In [ ]:
X_test.shape

(2, 4)

предобработка:

Приведите тексты к нижнему регистру (text.lower())

Замените все, кроме букв и цифр, на пробелы — это облегчит дальнейшее разделение текста на слова.


In [ ]:
X_train['FullDescription'] = X_train['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)
X_test['FullDescription'] = X_train['FullDescription'].str.lower().replace('[^a-zA-Z0-9]', ' ', regex = True)

In [ ]:
X_train.head()

,FullDescription,LocationNormalized,ContractTime,SalaryNormalized
0,international sales manager london k ...,London,permanent,33000
1,an ideal opportunity for an individual that ha...,London,permanent,50000
2,online content and brand manager luxury reta...,South East London,permanent,40000
3,a great local marketleader is seeking a perman...,Dereham,permanent,22500
4,registered nurse rgn nursing home for young...,Sutton Coldfield,NaN,20355


Примените TfidfVectorizer для преобразования текстов в векторы признаков. Оставьте только те слова, которые встречаются хотя бы в 5 объектах (параметр min_df у TfidfVectorizer).

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [ ]:
vec = TfidfVectorizer(min_df=5)

In [ ]:
X_train_text = vec.fit_transform(X_train["FullDescription"])

In [ ]:
X_test_text = vec.transform(X_test["FullDescription"])

Замените пропуски в столбцах LocationNormalized и ContractTime на специальную строку 'nan'. Код для этого был приведен выше.

In [ ]:
X_train['LocationNormalized'].fillna('nan', inplace=True)
X_train['ContractTime'].fillna('nan', inplace=True)
X_test['LocationNormalized'].fillna('nan', inplace=True)
X_test['ContractTime'].fillna('nan', inplace=True)

Примените DictVectorizer для получения one-hot-кодирования признаков LocationNormalized и ContractTime.

In [ ]:
from sklearn.feature_extraction import DictVectorizer
enc = DictVectorizer()
X_train_categ = enc.fit_transform(X_train[['LocationNormalized', 'ContractTime']].to_dict('records'))
X_test_categ = enc.transform(X_test[['LocationNormalized', 'ContractTime']].to_dict('records'))

Объедините все полученные признаки в одну матрицу "объекты-признаки". Обратите внимание, что матрицы для текстов и категориальных признаков являются разреженными. Для объединения их столбцов нужно воспользоваться функцией scipy.sparse.hstack.

In [ ]:
from scipy.sparse import hstack

In [ ]:
X_train = hstack([X_train_text, X_train_categ])

In [ ]:
X_test = hstack([X_test_text, X_test_categ])

Обучите гребневую регрессию с параметрами alpha=1 и random_state=241. Целевая переменная записана в столбце SalaryNormalized.

In [ ]:
from sklearn.linear_model import Ridge

In [ ]:
model = Ridge(alpha=1, random_state=241)
model.fit(X_train, y_train)

Ridge(alpha=1, random_state=241)

In [ ]:
y_test = model.predict(X_test)

In [ ]:
y_test

array([40024.8957201 , 35999.95601328])

###Задание по программированию: Составление фондового индекса

работать с методом главных компонент

использовать его для вычисления улучшенного индекса Доу-Джонса

In [2]:
from google.colab import files

In [4]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA

In [3]:
uploaded = files.upload()

Saving close_prices.csv to close_prices.csv


In [5]:
df = pd.read_csv("close_prices.csv")
df.head()

,date,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,2013-09-23,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,2013-09-24,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,2013-09-25,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,2013-09-26,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,2013-09-27,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [9]:
X = df.loc[:,'AXP':]

In [10]:
X.head()

,AXP,BA,CAT,CSCO,CVX,DD,DIS,GE,GS,HD,...,PFE,PG,T,TRV,UNH,UTX,V,VZ,WMT,XOM
0,76.440002,117.510002,85.029999,24.270000,125.519997,59.409999,64.750000,24.280001,165.250000,75.910004,...,28.799999,79.279999,34.220001,86.379997,71.820000,109.419998,196.240005,47.980000,76.419998,87.750000
1,76.070000,119.000000,85.110001,24.139999,124.489998,59.319997,64.320000,24.320000,162.970001,76.040001,...,28.709999,78.620003,34.090000,85.870003,72.320000,110.000000,193.339996,47.270000,75.750000,87.360001
2,75.989998,118.510002,84.500000,24.430000,124.070000,59.319997,64.449997,24.230000,162.309998,75.519997,...,28.490000,77.720001,34.049999,85.980003,71.980003,109.260002,191.559998,46.950001,74.650002,87.139999
3,76.320000,119.379997,84.199997,23.770000,123.489998,59.509996,65.239998,24.250000,162.289993,76.070000,...,28.520000,78.050003,34.230000,85.830002,72.160004,109.660004,193.559998,47.669998,74.620003,87.070000
4,75.889999,118.739998,83.800003,23.330000,122.639999,59.009995,65.190002,24.049999,159.850006,75.959999,...,28.879999,77.209999,33.980000,85.410004,71.989998,109.360001,193.050003,47.000000,74.360001,86.900002


In [11]:
pca = PCA(n_components=10)
pca.fit(X)

PCA(n_components=10)

In [12]:
pca.explained_variance_ratio_

array([0.73897118, 0.11007169, 0.04995088, 0.0287492 , 0.02215448,
       0.01931577, 0.00674853, 0.00614091, 0.00320594, 0.00305611])

In [16]:
pca.components_;

In [14]:
sum = 0
for i in range(10):
  if sum < 0.9:
    sum += pca.explained_variance_ratio_[i]
  else:
    print(i)
    print(sum)
    break

4
0.92774295378364


In [18]:
X_new = pca.transform(X)

In [19]:
X_new

array([[-50.90240358,  17.63167724,   7.7360209 , ...,   3.55657041,
         -5.82197358,  -1.72604005],
       [-52.84690919,  19.14690749,   7.27254551, ...,   3.43259929,
         -5.63318106,  -2.0122316 ],
       [-54.61443917,  19.92735811,   8.10995831, ...,   4.17520282,
         -5.38828678,  -2.75269933],
       ...,
       [ 83.79819948,  16.95241334,  -8.25265611, ...,   2.92390381,
         -4.59543822,  -2.99033503],
       [ 79.19578947,  19.14436008,  -6.93262976, ...,   3.40475421,
         -3.66793327,  -3.57607629],
       [ 82.96885933,  13.13513974,  -9.07644299, ...,   3.75100423,
         -4.79172198,  -3.03918865]])

In [29]:
X0 = pd.DataFrame(pca.transform(X))[0]
X0.head()

0   -50.902404
1   -52.846909
2   -54.614439
3   -52.600566
4   -52.370123
Name: 0, dtype: float64

In [23]:
uploaded = files.upload()

Saving djia_index.csv to djia_index.csv


In [24]:
djia = pd.read_csv('djia_index.csv')

In [25]:
djia.head()

,date,^DJI
0,2013-09-23,15401.379883
1,2013-09-24,15334.589844
2,2013-09-25,15273.259766
3,2013-09-26,15328.299805
4,2013-09-27,15258.240234


In [30]:
corr = np.corrcoef(X0, djia["^DJI"])

In [31]:
corr

array([[1.        , 0.90965222],
       [0.90965222, 1.        ]])

In [36]:
X.columns[np.argmax(pca.components_[0])]

'V'